## Data Cleaning for Biodiversity in National Parks

In [214]:
#libraries
import pandas as pd
pd.set_option('display.max_rows', None)

In [215]:
#read cvs and look at head to get a feel for data
observe = pd.read_csv('observations.csv')
print(observe.head())
print(observe.shape)
species = pd.read_csv('species_info.csv')
print(species.head())
print(species.shape)

            scientific_name                            park_name  observations
0        Vicia benghalensis  Great Smoky Mountains National Park            68
1            Neovison vison  Great Smoky Mountains National Park            77
2         Prunus subcordata               Yosemite National Park           138
3      Abutilon theophrasti                  Bryce National Park            84
4  Githopsis specularioides  Great Smoky Mountains National Park            85
(23296, 3)
  category                scientific_name  \
0   Mammal  Clethrionomys gapperi gapperi   
1   Mammal                      Bos bison   
2   Mammal                     Bos taurus   
3   Mammal                     Ovis aries   
4   Mammal                 Cervus elaphus   

                                        common_names conservation_status  
0                           Gapper's Red-Backed Vole                 NaN  
1                              American Bison, Bison                 NaN  
2  Aurochs, Aurochs

Check for duplicates in each datafram prior to merging. Also check for duplicates in the scientific name of the species data frame separate from the conservation status and common names.

In [216]:
#check for duplicates in observations (suppressed)
#print(observe[observe.duplicated(keep=False)==True].value_counts())
#remove those duplicates
observe.drop_duplicates(inplace=True)
#check for Nan Values
print(observe.isna().value_counts())
#remove 'National Park' from park_name column
observe['park_name']=observe['park_name'].replace(' National Park', '', regex=True)

scientific_name  park_name  observations
False            False      False           23281
dtype: int64


Investigate data issues in species csv. Look for and correct duplicates, missing values and conflicting data. 

In [217]:
#check for duplicates in species
print('Fully duplicated values in species')
print(species.duplicated().value_counts())
#check for species duplicates that vary only by conservation status
print("Species items that differ only by conservation status")
print(species.duplicated(subset=['scientific_name','category', 'common_names']).value_counts())
print(species[species.duplicated(subset=['scientific_name','category', 'common_names'], keep = False)==True])
#Remove row 560, as Oncorhynchus mykiss are a threatened species, and 3020, as Canis lupus legal status currently list them as Endangered. 
species.drop_duplicates(subset=['scientific_name','category', 'common_names'], keep = 'last', inplace= True)
print(species.duplicated(subset=['scientific_name','category', 'common_names']).value_counts())
#fill species['conservation_status'] values that are missing with 'No Status'
species.fillna(value={'conservation_status': 'No Status'}, inplace=True)
#check for species duplicates that vary only by common name
print("Species items that differ only by common names")
print(species.duplicated(subset=['scientific_name','category', 'conservation_status']).value_counts())
#group then aggregate
species= species.groupby(['scientific_name','category', 'conservation_status']).agg({'common_names': lambda x: ', '.join(tuple(x.tolist()))})
species.reset_index(inplace=True)
#print("check aggregated values with spot check of common names for canis lupis (at least on only had gray wolf.)")
print(species[species['scientific_name']=='Canis lupus'])
#It looks like there are duplicate items in the values of species_agg['common_names'] now, so I'll need to remove those. 
species['common_names']= species['common_names'].apply(lambda x: ', '.join(set(x.split(', '))))
print(species[species['scientific_name']=='Canis lupus'])

Fully duplicated values in species
False    5824
dtype: int64
Species items that differ only by conservation status
False    5822
True        2
dtype: int64
     category      scientific_name     common_names conservation_status
560      Fish  Oncorhynchus mykiss    Rainbow Trout                 NaN
3020   Mammal          Canis lupus  Gray Wolf, Wolf         In Recovery
3283     Fish  Oncorhynchus mykiss    Rainbow Trout          Threatened
4448   Mammal          Canis lupus  Gray Wolf, Wolf          Endangered
False    5822
dtype: int64
Species items that differ only by common names
False    5541
True      281
dtype: int64
    scientific_name category conservation_status                common_names
804     Canis lupus   Mammal          Endangered  Gray Wolf, Gray Wolf, Wolf
    scientific_name category conservation_status     common_names
804     Canis lupus   Mammal          Endangered  Wolf, Gray Wolf


Since the conservation status values are do not seem to include a 'least concern' category, it's likely that some of the the NaN values in this list are species that do not have a conservation status at this time. Since there's no way to confirm that, for now we've replaced those with the status 'No Status'. Many of the items without a listing are vascular plants, these are less likely to be tracked on the conservation status list as well. 

There are also 281 items that are completely duplicated apart from the common name. These are likely not unique observations, so we're going to drop rows that are duplicates apart from the common names from our dataset. 

The two clean csv's can now be joined on scientific name, outer. 

In [218]:
#merge the two csv, outer, keeping all rows
biodiverse = pd.merge(observe, species, how='outer', on='scientific_name')

In [219]:
#check data types and possible issues
print(biodiverse.shape)
print(biodiverse.isna().sum())
print(biodiverse.dtypes)
print(biodiverse.nunique())
print(biodiverse.head())

(23281, 6)
scientific_name        0
park_name              0
observations           0
category               0
conservation_status    0
common_names           0
dtype: int64
scientific_name        object
park_name              object
observations            int64
category               object
conservation_status    object
common_names           object
dtype: object
scientific_name        5541
park_name                 4
observations            304
category                  7
conservation_status       5
common_names           5236
dtype: int64
      scientific_name              park_name  observations        category  \
0  Vicia benghalensis  Great Smoky Mountains            68  Vascular Plant   
1  Vicia benghalensis               Yosemite           148  Vascular Plant   
2  Vicia benghalensis            Yellowstone           247  Vascular Plant   
3  Vicia benghalensis                  Bryce           104  Vascular Plant   
4      Neovison vison  Great Smoky Mountains            77   

We can now see that we're looking at data from four National Parks where they've observed 5541 unique species. The data has been cleaned and tidied, so now we can save it to a new csv, 'biodiversity_data.csv'

In [220]:
biodiverse.to_csv('biodiversity_data.csv', index=False)